# Deep Feed Forward Networks

In [1]:
import numpy as np 
import tensorflow as tf

In [2]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

## Network Construction Phase

In [3]:
num_inputs = 28*28 # MNIST
h1 = 300 # hidden layer 1,2 size
h2 = 100 
num_out = 10

### Set Placeholders

In [140]:
reset_graph()

In [141]:
X = tf.placeholder(tf.float32, shape=(None, num_inputs), name='X')
y = tf.placeholder(tf.int64, shape=(None, 10), name='y')

### Set Function For Neuron Layer

In [142]:
def neuron_layer(X, num_neurons, name, activation= None):
    with tf.name_scope(name):
        
        num_inputs = int(X.get_shape()[1])
        variance = 2 / np.sqrt(num_inputs)
        
        W = tf.Variable(tf.truncated_normal((num_inputs, num_neurons), stddev=variance), name='weights')
        b = tf.Variable(tf.zeros([num_neurons]), name='bias')
        
        z = tf.add(tf.matmul(X, W), b)
        
        if activation is None:
            return z

        activation = activation.lower()
        if activation == 'relu':
            return tf.nn.relu(z)
        elif activation == 'sigmoid':
            return tf.nn.sigmoid(z)
        elif activation == 'softmax':
            return tf.nn.softmax(z)

### Create all layers of the network

In [143]:
with tf.name_scope('dnn'):
    hidden1 = neuron_layer(X, h1, name='hidden1', activation='ReLU')
    hidden2 = neuron_layer(hidden1, h2, name='hidden2',activation='ReLU')
    out = neuron_layer(hidden2, num_out, name='out', activation='softmax')

### Define Loss

In [144]:
with tf.name_scope('loss'):
    entropy = tf.losses.softmax_cross_entropy(onehot_labels=y, logits=out)
    loss = tf.reduce_mean(entropy, name='loss')

### Define optimizer

In [11]:
eta = 1e-2

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=eta)
    train_step = optimizer.minimize(loss)

### Set Evaluation Method

In [12]:
with tf.name_scope('eval'):

    predictions = tf.equal(tf.argmax(out, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(predictions, tf.float32))

### Define Initializer and Saver

In [13]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

## LOAD MNIST AND TRAIN NETWORK

In [15]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./tmp/data')

Extracting ./tmp/data/train-images-idx3-ubyte.gz
Extracting ./tmp/data/train-labels-idx1-ubyte.gz
Extracting ./tmp/data/t10k-images-idx3-ubyte.gz
Extracting ./tmp/data/t10k-labels-idx1-ubyte.gz


### Train

In [16]:
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(30):
        for iteration in range(mnist.train.num_examples // batch_size):
            
            # get next batch
            X_batch_temp, y_batch_temp = mnist.train.next_batch(batch_size)
            X_batch = np.reshape(X_batch_temp, [-1, 784])
            y_batch = np.eye(10)[y_batch_temp] # numpy one-hot encoding
            
            # execute update step 
            sess.run(train_step, feed_dict={X: X_batch, y: y_batch})
            
        # Evaluate accuracy on test set
        test_acc = sess.run(accuracy, feed_dict={X: mnist.test.images, y:np.eye(10)[mnist.test.labels]})
        
        print(f'Test set accuracy in epoch {epoch+1}: {round(test_acc*100, 2)}%')
            
    # save model
    save_path = saver.save(sess, './models/final_model.ckpt')

Test set accuracy in epoch 1: 64.11%
Test set accuracy in epoch 2: 66.02%
Test set accuracy in epoch 3: 66.5%
Test set accuracy in epoch 4: 66.78%
Test set accuracy in epoch 5: 67.0%
Test set accuracy in epoch 6: 67.23%
Test set accuracy in epoch 7: 67.35%
Test set accuracy in epoch 8: 67.49%
Test set accuracy in epoch 9: 67.7%
Test set accuracy in epoch 10: 67.74%
Test set accuracy in epoch 11: 67.89%
Test set accuracy in epoch 12: 67.89%
Test set accuracy in epoch 13: 67.98%
Test set accuracy in epoch 14: 68.03%
Test set accuracy in epoch 15: 68.06%
Test set accuracy in epoch 16: 68.07%
Test set accuracy in epoch 17: 68.15%
Test set accuracy in epoch 18: 68.22%
Test set accuracy in epoch 19: 68.22%
Test set accuracy in epoch 20: 77.22%
Test set accuracy in epoch 21: 77.52%
Test set accuracy in epoch 22: 77.58%
Test set accuracy in epoch 23: 77.69%
Test set accuracy in epoch 24: 77.8%
Test set accuracy in epoch 25: 77.79%
Test set accuracy in epoch 26: 77.91%
Test set accuracy in epoc

## He Initialization + ELU activation function

In [68]:
reset_graph()

In [18]:
he_elu_X = tf.placeholder(tf.float32, shape=(None, num_inputs), name='he_elu_X')
he_elu_y = tf.placeholder(tf.int64, shape=(None, 10), name='he_elu_y')

In [26]:
def he_elu_neuron_layer(X, num_neurons, name, activation):
    with tf.name_scope(name):
        
        num_inputs = int(X.get_shape()[1])
        
        initializer = tf.contrib.layers.variance_scaling_initializer()
        W = tf.Variable(initializer((num_inputs, num_neurons)), name='weights')
        b = tf.Variable(tf.zeros([num_neurons]), name='he_elu_bias')
        
        z = tf.add(tf.matmul(X, W), b)
        
        if activation is None:
            return z

        activation = activation.lower()
        if activation == 'elu':
            return tf.nn.elu(z)
        elif activation == 'softmax':
            return tf.nn.softmax(z)

In [27]:
with tf.name_scope('he_elu_dnn'):
    he_elu_hidden1 = he_elu_neuron_layer(he_elu_X, h1, name='he_elu_hidden1', activation='elu')
    he_elu_hidden2 = he_elu_neuron_layer(he_elu_hidden1, h2, name='he_elu_hidden2',activation='elu')
    he_elu_out = he_elu_neuron_layer(he_elu_hidden2, num_out, name='he_elu_out', activation='softmax')

In [29]:
with tf.name_scope('he_elu_loss'):
    he_elu_entropy = tf.losses.softmax_cross_entropy(onehot_labels=he_elu_y, logits=he_elu_out)
    he_elu_loss = tf.reduce_mean(he_elu_entropy, name='he_elu_loss')

In [30]:
eta = 1e-2

with tf.name_scope('he_elu_train'):
    he_elu_optimizer = tf.train.GradientDescentOptimizer(learning_rate=eta)
    he_elu_train_step = he_elu_optimizer.minimize(he_elu_loss)

In [31]:
with tf.name_scope('he_elu_eval'):

    he_elu_predictions = tf.equal(tf.argmax(he_elu_out, 1), tf.argmax(he_elu_y, 1))
    he_elu_accuracy = tf.reduce_mean(tf.cast(he_elu_predictions, tf.float32))

In [33]:
he_elu_init = tf.global_variables_initializer()
he_elu_saver = tf.train.Saver()

In [35]:
batch_size = 50

with tf.Session() as sess:
    he_elu_init.run()
    for epoch in range(30):
        for iteration in range(mnist.train.num_examples // batch_size):
            
            # get next batch
            X_batch_temp, y_batch_temp = mnist.train.next_batch(batch_size)
            X_batch = np.reshape(X_batch_temp, [-1, 784])
            y_batch = np.eye(10)[y_batch_temp] # numpy one-hot encoding
            
            # execute update step 
            sess.run(he_elu_train_step, feed_dict={he_elu_X: X_batch, he_elu_y: y_batch})
            
        # Evaluate accuracy on test set
        he_elu_test_acc = sess.run(he_elu_accuracy, feed_dict={he_elu_X: mnist.test.images, he_elu_y:np.eye(10)[mnist.test.labels]})
        
        print(f'Test set accuracy in epoch {epoch+1}: {round(he_elu_test_acc*100, 2)}%')
            
    # save model
    save_path = he_elu_saver.save(sess, './models/he_elu_final_model.ckpt')

Test set accuracy in epoch 1: 72.8%
Test set accuracy in epoch 2: 76.03%
Test set accuracy in epoch 3: 81.1%
Test set accuracy in epoch 4: 82.04%
Test set accuracy in epoch 5: 82.61%
Test set accuracy in epoch 6: 82.83%
Test set accuracy in epoch 7: 90.48%
Test set accuracy in epoch 8: 91.07%
Test set accuracy in epoch 9: 91.19%
Test set accuracy in epoch 10: 91.54%
Test set accuracy in epoch 11: 91.77%
Test set accuracy in epoch 12: 91.96%
Test set accuracy in epoch 13: 92.11%
Test set accuracy in epoch 14: 92.28%
Test set accuracy in epoch 15: 92.44%
Test set accuracy in epoch 16: 92.54%
Test set accuracy in epoch 17: 92.69%
Test set accuracy in epoch 18: 92.68%
Test set accuracy in epoch 19: 92.71%
Test set accuracy in epoch 20: 92.78%
Test set accuracy in epoch 21: 92.91%
Test set accuracy in epoch 22: 92.97%
Test set accuracy in epoch 23: 93.07%
Test set accuracy in epoch 24: 93.05%
Test set accuracy in epoch 25: 93.22%
Test set accuracy in epoch 26: 93.21%
Test set accuracy in ep

## Adding Batch normalization

In [89]:
reset_graph()

In [90]:
bn_X = tf.placeholder(tf.float32, shape=(None, num_inputs), name='bn_X')
bn_y = tf.placeholder(tf.int64, shape=(None, 10), name='bn_y')

In [110]:
def bn_neuron_layer(X, num_neurons, name, activation):
    with tf.name_scope(name):
        
        num_inputs = int(X.get_shape()[1])
        
        initializer = tf.contrib.layers.variance_scaling_initializer()
        
        if activation == 'elu':
            dense = tf.contrib.layers.fully_connected(X, num_neurons, activation_fn=tf.nn.elu, weights_initializer=initializer, scope=name, reuse=tf.AUTO_REUSE)
            bn_dense = tf.contrib.layers.batch_norm(dense, scope=name, reuse=tf.AUTO_REUSE)
            
            return bn_dense
        
        elif activation == 'softmax':
            dense = tf.contrib.layers.fully_connected(X, num_neurons, activation_fn=tf.nn.softmax, weights_initializer=initializer, scope=name, reuse=tf.AUTO_REUSE)
            
            return dense

In [111]:
with tf.name_scope('bn_dnn'):
    bn_hidden1 = bn_neuron_layer(bn_X, h1, name='bn_hidden1', activation='elu')
    bn_hidden2 = bn_neuron_layer(bn_hidden1, h2, name='bn_hidden2',activation='elu')
    bn_out = bn_neuron_layer(bn_hidden2, num_out, name='bn_out', activation='softmax')

In [74]:
with tf.name_scope('bn_loss'):
    bn_entropy = tf.losses.softmax_cross_entropy(onehot_labels=bn_y, logits=bn_out)
    bn_loss = tf.reduce_mean(bn_entropy, name='bn_loss')

In [75]:
eta = 1e-2

with tf.name_scope('bn_train'):
    bn_optimizer = tf.train.GradientDescentOptimizer(learning_rate=eta)
    bn_train_step = bn_optimizer.minimize(bn_loss)

In [76]:
with tf.name_scope('bn_eval'):

    bn_predictions = tf.equal(tf.argmax(bn_out, 1), tf.argmax(bn_y, 1))
    bn_accuracy = tf.reduce_mean(tf.cast(bn_predictions, tf.float32))

In [77]:
bn_init = tf.global_variables_initializer()
bn_saver = tf.train.Saver()

In [78]:
batch_size = 50

with tf.Session() as sess:
    bn_init.run()
    for epoch in range(30):
        for iteration in range(mnist.train.num_examples // batch_size):
            
            # get next batch
            X_batch_temp, y_batch_temp = mnist.train.next_batch(batch_size)
            X_batch = np.reshape(X_batch_temp, [-1, 784])
            y_batch = np.eye(10)[y_batch_temp] # numpy one-hot encoding
            
            # execute update step 
            sess.run(bn_train_step, feed_dict={bn_X: X_batch, bn_y: y_batch})
            
        # Evaluate accuracy on test set
        bn_test_acc = sess.run(bn_accuracy, feed_dict={bn_X: mnist.test.images, bn_y:np.eye(10)[mnist.test.labels]})
        
        print(f'Test set accuracy in epoch {epoch+1}: {round(bn_test_acc*100, 2)}%')
            
    # save model
    save_path = bn_saver.save(sess, './models/bn_final_model.ckpt')

Test set accuracy in epoch 1: 88.45%
Test set accuracy in epoch 2: 90.8%
Test set accuracy in epoch 3: 91.58%
Test set accuracy in epoch 4: 92.15%
Test set accuracy in epoch 5: 92.47%
Test set accuracy in epoch 6: 92.85%
Test set accuracy in epoch 7: 92.91%
Test set accuracy in epoch 8: 93.17%
Test set accuracy in epoch 9: 93.39%
Test set accuracy in epoch 10: 93.54%
Test set accuracy in epoch 11: 93.76%
Test set accuracy in epoch 12: 93.98%
Test set accuracy in epoch 13: 94.15%
Test set accuracy in epoch 14: 94.34%
Test set accuracy in epoch 15: 94.47%
Test set accuracy in epoch 16: 94.64%
Test set accuracy in epoch 17: 94.74%
Test set accuracy in epoch 18: 94.86%
Test set accuracy in epoch 19: 94.99%
Test set accuracy in epoch 20: 95.06%
Test set accuracy in epoch 21: 95.12%
Test set accuracy in epoch 22: 95.2%
Test set accuracy in epoch 23: 95.26%
Test set accuracy in epoch 24: 95.36%
Test set accuracy in epoch 25: 95.48%
Test set accuracy in epoch 26: 95.46%
Test set accuracy in ep

## Better Optimiser

In [129]:
reset_graph()

In [130]:
adam_X = tf.placeholder(tf.float32, shape=(None, num_inputs), name='adam_X')
adam_y = tf.placeholder(tf.int64, shape=(None, 10), name='adam_y')

In [131]:
with tf.name_scope('adam_dnn'): # Using bn_neuron_layer because it does not change the netowrk feedforward is the same
    adam_hidden1 = bn_neuron_layer(adam_X, h1, name='adam_hidden1', activation='elu')
    adam_hidden2 = bn_neuron_layer(adam_hidden1, h2, name='adam_hidden2',activation='elu')
    adam_out = bn_neuron_layer(adam_hidden2, num_out, name='adam_out', activation='softmax')

In [135]:
with tf.name_scope('adam_loss'):
    adam_entropy = tf.losses.softmax_cross_entropy(onehot_labels=adam_y, logits=adam_out)
    adam_loss = tf.reduce_mean(adam_entropy, name='adam_loss')

In [136]:
eta = 1e-2

with tf.name_scope('adam_train'):
    adam_optimizer = tf.train.AdamOptimizer(learning_rate=eta)
    adam_train_step = adam_optimizer.minimize(loss)

In [137]:
with tf.name_scope('adam_eval'):

    adam_predictions = tf.equal(tf.argmax(adam_out, 1), tf.argmax(adam_y, 1))
    adam_accuracy = tf.reduce_mean(tf.cast(adam_predictions, tf.float32))

In [138]:
adam_init = tf.global_variables_initializer()
adam_saver = tf.train.Saver()

In [139]:
batch_size = 50

with tf.Session() as sess:
    adam_init.run()
    for epoch in range(30):
        for iteration in range(mnist.train.num_examples // batch_size):
            
            # get next batch
            X_batch_temp, y_batch_temp = mnist.train.next_batch(batch_size)
            X_batch = np.reshape(X_batch_temp, [-1, 784])
            y_batch = np.eye(10)[y_batch_temp] # numpy one-hot encoding
            
            # execute update step 
            sess.run(adam_train_step, feed_dict={adam_X: X_batch, adam_y: y_batch})
            
        # Evaluate accuracy on test set
        adam_test_acc = sess.run(adam_accuracy, feed_dict={adam_X: mnist.test.images, adam_y:np.eye(10)[mnist.test.labels]})
        
        print(f'Test set accuracy in epoch {epoch+1}: {round(adam_test_acc*100, 2)}%')
            
    # save model
    save_path = adam_saver.save(sess, './models/adam_final_model.ckpt')

Test set accuracy in epoch 1: 95.22%
Test set accuracy in epoch 2: 95.81%
Test set accuracy in epoch 3: 95.95%
Test set accuracy in epoch 4: 96.52%
Test set accuracy in epoch 5: 96.45%
Test set accuracy in epoch 6: 96.68%
Test set accuracy in epoch 7: 96.88%
Test set accuracy in epoch 8: 96.61%
Test set accuracy in epoch 9: 96.84%
Test set accuracy in epoch 10: 96.97%
Test set accuracy in epoch 11: 96.78%
Test set accuracy in epoch 12: 96.86%
Test set accuracy in epoch 13: 97.34%
Test set accuracy in epoch 14: 97.12%
Test set accuracy in epoch 15: 97.16%
Test set accuracy in epoch 16: 97.29%
Test set accuracy in epoch 17: 96.83%
Test set accuracy in epoch 18: 97.05%
Test set accuracy in epoch 19: 97.24%
Test set accuracy in epoch 20: 97.41%
Test set accuracy in epoch 21: 97.42%
Test set accuracy in epoch 22: 97.21%
Test set accuracy in epoch 23: 97.25%
Test set accuracy in epoch 24: 97.46%
Test set accuracy in epoch 25: 97.51%
Test set accuracy in epoch 26: 97.12%
Test set accuracy in 

## Decaying the Learning Rate